In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from math import sqrt
from datetime import datetime

In [2]:
cnx = sqlite3.connect('data/database.sqlite')
player_data = pd.read_sql_query("SELECT * FROM Player_Attributes", cnx)
player_data.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [3]:
categorical_attributes = ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']

numercial_attributes = ['potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes']

target_attribute = ['overall_rating']

In [4]:
player_data.shape

(183978, 42)

In [5]:
player_data.dropna(thresh=5, inplace=True)

In [6]:
player_data.shape

(183142, 42)

In [7]:
from sklearn.base import TransformerMixin, BaseEstimator


class Clean_and_Merge(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        categorical_attributes = ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']

        numercial_attributes = ['potential', 'crossing', 'finishing', 'heading_accuracy',
            'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
            'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
            'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
            'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
            'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
            'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
            'gk_reflexes']
        player_id = df['player_fifa_api_id'].unique()
        
        new_df = pd.DataFrame()

        for i in player_id:
            index = np.where(df['player_fifa_api_id'] == i)[0]

            temp_num = df.iloc[index][numercial_attributes].mean()
            temp_cat = df.iloc[index][categorical_attributes].mode().iloc[0]

            temp_df = pd.DataFrame(data=[temp_num.values], columns=temp_num.index)

            temp_df[temp_cat.index] = temp_cat.values

            new_df = new_df.append(temp_df)
            
        
        to_drop = ['norm', 'y', 'le', 'stoc']

        for i in to_drop:
            new_df['attacking_work_rate'].replace(i, np.nan, inplace=True)
            
        to_drop = ['ormal', 'ean', 'es', 'tocky', '_0', 'o']

        for i in to_drop:
            new_df['defensive_work_rate'].replace(i, np.nan, inplace=True)
            
            
        low_class = ['0', '1', '2']
        medium_class = ['3', '4', '5', '6']
        high_class = ['7', '8', '9']

        for i in low_class:
            new_df['defensive_work_rate'].replace(i, 'low', inplace=True)

        for i in medium_class:
            new_df['defensive_work_rate'].replace(i, 'medium', inplace=True)

        for i in high_class:
            new_df['defensive_work_rate'].replace(i, 'high', inplace=True)
            
            
        return new_df.reset_index(drop=True)
    
    

class Clean_and_Merge_Target(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, df, y=None):
        return self
    
    def transform(self, df, y=None):
        df.dropna(thresh=10, inplace=True)
        player_id = df['player_fifa_api_id'].unique()
        
        new_df = pd.DataFrame()

        for i in player_id:
            index = np.where(df['player_fifa_api_id'] == i)[0]

            temp_num = df.iloc[index][target_attribute].mean()

            temp_df = pd.DataFrame(data=[temp_num.values], columns=temp_num.index)

            new_df = new_df.append(temp_df)
            
        
        return new_df.reset_index(drop=True)

In [8]:
def generate_file_name(filename):
    now = datetime.now()
    current = now.strftime("%d_%m_%Y-%H_%M_%S")
    
    return now.strftime("%d_%m_%Y-%H_%M_%S") + '_' + filename 

In [9]:
from Clean_and_Merge import Clean_and_Merge

custom_attribute_object = Clean_and_Merge()

merged_player_data = custom_attribute_object.fit_transform(player_data)
merged_player_data.head()

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,67.600000,48.600000,43.600000,70.600000,60.600000,43.600000,50.600000,44.600000,38.600000,63.600000,...,66.000000,67.800000,5.600000,10.600000,9.600000,7.600000,7.600000,right,medium,medium
1,74.484848,70.787879,49.454545,52.939394,62.272727,29.151515,61.090909,61.878788,62.121212,63.242424,...,68.787879,71.515152,12.181818,8.666667,14.242424,10.363636,12.909091,left,medium,medium
2,74.192308,68.115385,57.923077,58.692308,65.115385,54.269231,69.038462,60.192308,55.615385,60.461538,...,21.115385,21.346154,14.038462,11.807692,17.730769,10.115385,13.500000,right,medium,medium
3,70.782609,57.217391,26.260870,69.260870,64.695652,47.782609,55.565217,37.782609,40.391304,60.826087,...,70.652174,68.043478,14.173913,11.173913,22.869565,11.173913,10.173913,right,medium,medium
4,74.680000,45.080000,38.840000,73.040000,64.760000,32.080000,50.600000,45.480000,26.360000,56.840000,...,76.040000,74.600000,8.280000,8.320000,24.920000,12.840000,11.920000,right,medium,medium


In [10]:
from Clean_and_Merge_Target import Clean_and_Merge_Target

merged_target_object = Clean_and_Merge_Target()
merged_target_data = merged_target_object.fit_transform(player_data)
merged_target_data.head()

,overall_rating
0,63.600000
1,66.969697
2,67.000000
3,69.086957
4,73.240000


In [11]:
merged_player_data

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,67.600000,48.600000,43.600000,70.600000,60.600000,43.600000,50.600000,44.600000,38.600000,63.600000,...,66.000000,67.800000,5.600000,10.600000,9.600000,7.600000,7.600000,right,medium,medium
1,74.484848,70.787879,49.454545,52.939394,62.272727,29.151515,61.090909,61.878788,62.121212,63.242424,...,68.787879,71.515152,12.181818,8.666667,14.242424,10.363636,12.909091,left,medium,medium
2,74.192308,68.115385,57.923077,58.692308,65.115385,54.269231,69.038462,60.192308,55.615385,60.461538,...,21.115385,21.346154,14.038462,11.807692,17.730769,10.115385,13.500000,right,medium,medium
3,70.782609,57.217391,26.260870,69.260870,64.695652,47.782609,55.565217,37.782609,40.391304,60.826087,...,70.652174,68.043478,14.173913,11.173913,22.869565,11.173913,10.173913,right,medium,medium
4,74.680000,45.080000,38.840000,73.040000,64.760000,32.080000,50.600000,45.480000,26.360000,56.840000,...,76.040000,74.600000,8.280000,8.320000,24.920000,12.840000,11.920000,right,medium,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,75.461538,42.000000,27.000000,75.153846,70.000000,30.000000,56.384615,49.000000,35.692308,54.538462,...,76.307692,72.692308,12.846154,12.769231,25.461538,11.384615,13.615385,right,low,medium
11056,71.625000,67.250000,46.750000,60.312500,63.125000,54.562500,61.187500,63.812500,61.750000,62.312500,...,65.250000,63.875000,9.000000,8.000000,5.000000,5.000000,8.000000,left,medium,medium
11057,72.857143,63.142857,44.571429,59.857143,66.857143,57.000000,66.857143,60.000000,52.571429,58.285714,...,64.285714,59.000000,7.142857,18.714286,42.428571,16.714286,16.714286,left,NaN,NaN
11058,78.125000,46.750000,43.000000,79.000000,58.875000,59.000000,36.000000,29.000000,41.375000,50.625000,...,73.250000,65.000000,14.000000,18.500000,41.625000,19.000000,20.000000,right,NaN,NaN


In [12]:
merged_target_data.shape

(11060, 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(merged_player_data, 
                                                    merged_target_data, 
                                                    test_size=0.20, 
                                                    random_state=101)

In [14]:
X_train.shape

(8848, 37)

In [15]:
X_test.shape

(2212, 37)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [17]:
num_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')), 
    ('num_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='most_frequent')), 
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_transformer = ColumnTransformer([
    ('num_transformer', num_pipeline, numercial_attributes),
    ('cat_transformer', cat_pipeline, categorical_attributes)
])

target_pipeline = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean', add_indicator=True)), 
    ('num_scaler', StandardScaler())
])

In [18]:
full_transformed_data = full_transformer.fit_transform(X_train)
full_transformed_data

array([[ 0.58197118,  0.65919831,  1.21870086, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.75999394,  0.5594177 ,  1.61461829, ...,  0.        ,
         1.        ,  0.        ],
       [-0.58987965, -0.87256068, -1.07515648, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.61114926, -0.49402403, -0.94457582, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.77446541, -0.15817229, -0.54870336, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.91323318,  0.97212564,  0.74132583, ...,  0.        ,
         1.        ,  0.        ]])

In [19]:
full_transformed_target = target_pipeline.fit_transform(pd.DataFrame(y_train))
full_transformed_target

array([[ 0.81114056],
       [ 1.00651404],
       [-0.11224976],
       ...,
       [ 1.23316027],
       [ 0.31232155],
       [ 0.88329677]])

In [20]:
import joblib
import os

In [21]:
dir_name = 'pipelines'
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)
    
    
joblib.dump(custom_attribute_object, os.path.join(dir_name, generate_file_name('custom_attribute_object.pkl')))
joblib.dump(merged_target_object, os.path.join(dir_name, generate_file_name('merged_target_object.pkl')))    
joblib.dump(full_transformer, os.path.join(dir_name, generate_file_name('full_transformer.pkl')))
joblib.dump(target_pipeline, os.path.join(dir_name, generate_file_name('target_pipeline.pkl')))

['pipelines/21_12_2022-20_01_08_target_pipeline.pkl']

In [22]:
full_transformed_data[0]

array([ 0.58197118,  0.65919831,  1.21870086, -0.15458867,  0.77378575,
        1.00827751,  1.71482358,  1.90742562,  1.4899623 ,  1.29099532,
        1.54551154,  0.15891902, -0.1266653 ,  0.95284264,  0.45336522,
        0.51015573,  0.51292227, -1.32160634, -1.32876564, -0.57434933,
        1.37964716, -1.21811061, -0.52329598,  0.59828634,  0.29267976,
        0.98687274, -1.30232533, -1.18465623, -1.68061165, -0.45261971,
       -0.43010749,  0.41935989,  0.02415765, -0.37595758,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  1.        ,  0.        ])

In [23]:
full_transformed_data.shape

(8848, 43)

In [24]:
full_transformed_target.shape

(8848, 1)

In [25]:
from sklearn.model_selection import cross_val_score

In [26]:
%%time

lin_reg = LinearRegression()

cross_val_score(estimator=lin_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)



CPU times: user 42.7 ms, sys: 131 ms, total: 174 ms
Wall time: 2.7 s


array([0.86573554, 0.87632791, 0.84637004, 0.86510455, 0.86442294,
       0.87188666, 0.86211904, 0.87657351, 0.86725609, 0.86585391])

In [27]:
%%time

tree_reg = DecisionTreeRegressor()

cross_val_score(estimator=tree_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

CPU times: user 16 ms, sys: 8.6 ms, total: 24.6 ms
Wall time: 748 ms


array([0.88566323, 0.87477967, 0.85499768, 0.85631288, 0.87081781,
       0.8714486 , 0.87272732, 0.86016746, 0.87308794, 0.85064654])

In [28]:
%%time

from sklearn.svm import SVR

svm_reg = SVR()

cross_val_score(estimator=svm_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

/Users/aayushman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/aayushman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/aayushman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/aayushman/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

CPU times: user 22.3 ms, sys: 14.5 ms, total: 36.8 ms
Wall time: 7.07 s


array([0.97685937, 0.97802457, 0.97273733, 0.97484917, 0.97245267,
       0.97649227, 0.97955977, 0.97637403, 0.97869964, 0.97274002])

In [29]:
%%time

from xgboost import XGBRegressor

xgb_reg = XGBRegressor(gpu_id=0, predictor='gpu_predictor')

cross_val_score(estimator=xgb_reg, X=full_transformed_data,
               y=full_transformed_target, cv=10, n_jobs=-1)

ModuleNotFoundError: No module named 'xgboost'

In [30]:
test_data = full_transformer.transform(X_test)
test_data

array([[ 1.0083062 ,  0.72221711,  1.49367349, ...,  0.        ,
         1.        ,  0.        ],
       [-1.2213958 ,  0.13374578,  0.28608535, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.91008926,  1.19957847,  0.04166657, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.96304828, -0.44522506,  0.64641407, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.5439789 ,  0.71530132, -1.33539389, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.05674455, -2.33776321, -1.88755753, ...,  0.        ,
         0.        ,  1.        ]])

In [31]:
test_target = target_pipeline.transform(pd.DataFrame(y_test))
test_target

array([[ 1.54347097],
       [-1.89001143],
       [ 0.82996392],
       ...,
       [-1.09001868],
       [ 0.43996745],
       [ 0.74575416]])

In [32]:
best_model_details = {}

In [33]:
lin_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = lin_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[lin_reg] = error

Error :  0.3578940182156812


In [34]:
tree_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = tree_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[tree_reg] = error

Error :  0.36456890303957873


In [35]:
svm_reg.fit(X=full_transformed_data, y=full_transformed_target.ravel())

y_pred = svm_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[svm_reg] = error

Error :  0.14702290899009138


In [36]:
full_transformed_target.ravel()

array([ 0.81114056,  1.00651404, -0.11224976, ...,  1.23316027,
        0.31232155,  0.88329677])

In [37]:
xgb_reg.fit(X=full_transformed_data, y=full_transformed_target)

y_pred = xgb_reg.predict(test_data)
y_pred = y_pred.reshape(-1, 1)

test_target = target_pipeline.transform(pd.DataFrame(y_test))

error = sqrt(mean_squared_error(y_pred=y_pred, y_true=test_target))
print('Error : ', error)

best_model_details[xgb_reg] = error

NameError: name 'xgb_reg' is not defined

In [38]:
best_model_details

{LinearRegression(): 0.3578940182156812,
 DecisionTreeRegressor(): 0.36456890303957873,
 SVR(): 0.14702290899009138}

In [39]:
min_value = list(best_model_details.values())[0]
my_model = ''

In [40]:
for key, value in best_model_details.items():
    if value < min_value:
        min_value = value
        my_model = key
        
print('Best performing model : {} \nError : {}'.format(my_model, min_value))

Best performing model : SVR() 
Error : 0.14702290899009138


In [41]:
dir_name = 'models'
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)
    
joblib.dump(my_model, os.path.join(dir_name, generate_file_name('best_model.pkl')))

['models/21_12_2022-20_02_20_best_model.pkl']